# Computer Vision and Image Processing - Lab Session 4

Content:
1. Introduction to Pytorch (from University of Amsterdam https://uvadlc.github.io)
2. Neural Networks with pytorch
3. Training loop

Contacts:

- Prof: Giuseppe Lisanti  (giuseppe.lisanti@unibo.it)
- Prof: Samuele Salti  (samuele.salti@unibo.it)
- Tutor: Adriano Cardace (adriano.cardace2@unibo.it)

Course:

- Website and notebooks will be available at https://virtuale.unibo.it

# Import Dependencies

In [ ]:
import math
import random
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
import time

# Runtime Settings

For this hands-on session and the next one, we are going to train on GPUs. To request a GPU for you virtual machine in Colab, you should change the Runtime settings. 

![nll](https://cdn.analyticsvidhya.com/wp-content/uploads/2020/03/uc14.png)

## The Basics of PyTorch

We will start with reviewing the very basic concepts of PyTorch. As a prerequisite, we recommend to be familiar with the `numpy` package as most machine learning frameworks are based on very similar concepts. If you are not familiar with numpy yet, don't worry: here is a [tutorial](https://numpy.org/devdocs/user/quickstart.html) to go through. 

So, let's start with importing PyTorch. The package is called `torch`, based on its original framework [Torch](http://torch.ch/). As a first step, we can check its version:

In [ ]:
import torch
print("Using torch", torch.__version__)

# Reproducibility
When **developing and debugging** Neural Networks is desirable to have a deterministic behaviour. For this reason, we are going to disable all the sources of randomness. Please note that completely reproducible results are not guaranteed across PyTorch releases, individual commits or different platforms. You can find more detailed information at this [page](https://pytorch.org/docs/stable/notes/randomness.html).
Please note that the flag `cudnn.benchmark = False` disable the auto-tuner that selects the optimal set of algorithms for your hardware and usually leads to slower runtime. 

In [ ]:
def fix_random(seed: int) -> None:
    """Fix all the possible sources of randomness.

    Args:
        seed: the seed to use. 
    """
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True        

fix_random(42)

# Intro to Pytorch!


### Tensors

Tensors are the PyTorch equivalent to Numpy arrays, with the addition to also have support for GPU acceleration (more on that later).
The name "tensor" is a generalization of concepts you already know. For instance, a vector is a 1-D tensor, and a matrix a 2-D tensor. When working with neural networks, we will use tensors of various shapes and number of dimensions.

Most common functions you know from numpy can be used on tensors as well. Actually, since numpy arrays are so similar to tensors, we can convert most tensors to numpy arrays (and back) but we don't need it too often.

#### Initialization

Let's first start by looking at different ways of creating a tensor. There are many possible options, the simplest one is to call `torch.Tensor` passing the desired shape as input argument:

In [ ]:
x = torch.Tensor(2, 3, 4)
print(x)

The function `torch.Tensor` allocates memory for the desired tensor, but reuses any values that have already been in the memory. To directly assign values to the tensor during initialization, there are many alternatives including:

* `torch.zeros`: Creates a tensor filled with zeros
* `torch.ones`: Creates a tensor filled with ones
* `torch.rand`: Creates a tensor with random values uniformly sampled between 0 and 1
* `torch.randn`: Creates a tensor with random values sampled from a normal distribution with mean 0 and variance 1
* `torch.arange`: Creates a tensor containing the values $N,N+1,N+2,...,M$
* `torch.Tensor` (input list): Creates a tensor from the list elements you provide

In [ ]:
# Create a tensor from a (nested) list
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

In [ ]:
# Create a tensor with random values between 0 and 1 with the shape [2, 3, 4]
x = torch.rand(2, 3, 4)
print(x)
print(x.shape)

#### Tensor to Numpy, and Numpy to Tensor

Tensors can be converted to numpy arrays, and numpy arrays back to tensors. To transform a numpy array into a tensor, we can use the function `torch.from_numpy`:

In [ ]:
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)

print("Numpy array:", np_arr)
print("PyTorch tensor:", tensor)

To transform a PyTorch tensor back to a numpy array, we can use the function `.numpy()` on tensors:

In [ ]:
tensor = torch.arange(4)
np_arr = tensor.numpy()

print("PyTorch tensor:", tensor)
print("Numpy array:", np_arr)

The conversion of tensors to numpy require the tensor to be on the CPU, and not the GPU (more on GPU support in a later section). In case you have a tensor on GPU, you need to call `.cpu()` on the tensor beforehand. Hence, you get a line like `np_arr = tensor.cpu().numpy()`.

#### Basic Operations


In [ ]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
y = x1 + x2

print("X1", x1)
print("X2", x2)
print("Y", y)

In [ ]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
print("X1 (before)", x1)
print("X2 (before)", x2)

x2.add_(x1)
print("X1 (after)", x1)
print("X2 (after)", x2)

In-place operations are usually marked with a underscore postfix (e.g. "add_" instead of "add").

Another common operation aims at changing the shape of a tensor. A tensor of size (2,3) can be re-organized to any other shape with the same number of elements (e.g. a tensor of size (6), or (3,2), ...). In PyTorch, this operation is called `view`:

In [ ]:
x = torch.arange(6)
print("X", x)

In [ ]:
x = x.view(2, 3)
print("X", x)

In [ ]:
x = x.permute(1, 0) # Swapping dimension 0 and 1
print("X", x)

### GPU support

In [ ]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

In [ ]:
x = torch.zeros(2, 3)
x = x.to(device)
print("X", x)

In case you have a GPU, you should now see the attribute `device='cuda:0'` being printed next to your tensor. The zero next to cuda indicates that this is the zero-th GPU device on your computer. PyTorch also supports multi-GPU systems, but this you will only need once you have very big networks to train (if interested, see the [PyTorch documentation](https://pytorch.org/docs/stable/distributed.html#distributed-basics)). We can also compare the runtime of a large matrix multiplication on the CPU with a operation on the GPU:

In [ ]:
x = torch.randn(5000, 5000)

## CPU version
start_time = time.time()
_ = torch.matmul(x, x)
end_time = time.time()
print(f"CPU time: {(end_time - start_time):6.5f}s")

## GPU version
x = x.to(device)
_ = torch.matmul(x, x)  # First operation to 'burn in' GPU
# CUDA is asynchronous, so we need to use different timing functions
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)
start.record()
_ = torch.matmul(x, x)
end.record()
torch.cuda.synchronize()  # Waits for everything to finish running on the GPU
print(f"GPU time: {0.001 * start.elapsed_time(end):6.5f}s")  # Milliseconds to seconds

### Dynamic Computation Graph and Backpropagation

One of the main reasons for using PyTorch in Deep Learning projects is that we can automatically get **gradients/derivatives** of functions that we define. We will mainly use PyTorch for implementing neural networks, and they are just fancy functions. If we use weight matrices in our function that we want to learn, then those are called the **parameters** or simply the **weights**.

If our neural network would output a single scalar value, we would talk about taking the **derivative**, but you will see that quite often we will have **multiple** output variables ("values"); in that case we talk about **gradients**. It's a more general term.

Given an input $\mathbf{x}$, we define our function by **manipulating** that input, usually by matrix-multiplications with weight matrices and additions with so-called bias vectors. As we manipulate our input, we are automatically creating a **computational graph**. This graph shows how to arrive at our output from our input. 
PyTorch is a **define-by-run** framework; this means that we can just do our manipulations, and PyTorch will keep track of that graph for us. Thus, we create a dynamic computation graph along the way.

So, to recap: the only thing we have to do is to compute the **output**, and then we can ask PyTorch to automatically get the **gradients**. 

> **Note:  Why do we want gradients?** Consider that we have defined a function, a neural net, that is supposed to compute a certain output $y$ for an input vector $\mathbf{x}$. We then define an **error measure** that tells us how wrong our network is; how bad it is in predicting output $y$ from input $\mathbf{x}$. Based on this error measure, we can use the gradients to **update** the weights $\mathbf{W}$ that were responsible for the output, so that the next time we present input $\mathbf{x}$ to our network, the output will be closer to what we want.

The first thing we have to do is to specify which tensors require gradients. By default, when we create a tensor, it does not require gradients.

In [ ]:
x = torch.ones((3,))
print(x.requires_grad)

We can change this for an existing tensor using the function `requires_grad_()` (underscore indicating that this is a in-place operation). Alternatively, when creating a tensor, you can pass the argument `requires_grad=True` to most initializers we have seen above.

In [ ]:
x.requires_grad_(True)
print(x.requires_grad)

In order to get familiar with the concept of a computation graph, we will create one for the following function:

$$y = \frac{1}{|x|}\sum_i \left[(x_i + 2)^2 + 3\right]$$

You could imagine that $x$ are our parameters, and we want to optimize (either maximize or minimize) the output $y$. For this, we want to obtain the gradients $\partial y / \partial \mathbf{x}$. For our example, we'll use $\mathbf{x}=[0,1,2]$ as our input.

In [ ]:
x = torch.arange(3, dtype=torch.float32, requires_grad=True) # Only float tensors can have gradients
print("X", x)

Now let's build the computation graph step by step. You can combine multiple operations in a single line, but we will separate them here to get a better understanding of how each operation is added to the computation graph.

In [ ]:
a = x + 2
b = a ** 2
c = b + 3
y = c.mean()
print("Y", y)

Using the statements above, we have created a computation graph that looks similar to the figure below:

pytorch_computation_graph.svg

We calculate $a$ based on the inputs $x$ and the constant $2$, $b$ is $a$ squared, and so on. The visualization is an abstraction of the dependencies between inputs and outputs of the operations we have applied.
Each node of the computation graph has automatically defined a function for calculating the gradients with respect to its inputs, `grad_fn`. You can see this when we printed the output tensor $y$. This is why the computation graph is usually visualized in the reverse direction (arrows point from the result to the inputs). We can perform backpropagation on the computation graph by calling the function `backward()` on the last output, which effectively calculates the gradients for each tensor that has the property `requires_grad=True`:

In [ ]:
y.backward()

`x.grad` will now contain the gradient $\partial y/ \partial \mathcal{x}$, and this gradient indicates how a change in $\mathbf{x}$ will affect output $y$ given the current input $\mathbf{x}=[0,1,2]$:

In [ ]:
print(x.grad)

We can also verify these gradients by hand. We will calculate the gradients using the chain rule, in the same way as PyTorch did it:

$$\frac{\partial y}{\partial x_i} = \frac{\partial y}{\partial c_i}\frac{\partial c_i}{\partial b_i}\frac{\partial b_i}{\partial a_i}\frac{\partial a_i}{\partial x_i}$$

Note that we have simplified this equation to index notation, and by using the fact that all operation besides the mean do not combine the elements in the tensor. The partial derivatives are:

$$
\frac{\partial a_i}{\partial x_i} = 1,\hspace{1cm}
\frac{\partial b_i}{\partial a_i} = 2\cdot a_i\hspace{1cm}
\frac{\partial c_i}{\partial b_i} = 1\hspace{1cm}
\frac{\partial y}{\partial c_i} = \frac{1}{3}
$$

Hence, with the input being $\mathbf{x}=[0,1,2]$, our gradients are $\partial y/\partial \mathbf{x}=[4/3,2,8/3]$. The previous code cell should have printed the same result.

# Let's build our first Neural Network!

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split, DataLoader, TensorDataset
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math
from timeit import default_timer as timer
from functools import partial

plt.rcParams['figure.figsize'] = (12.0, 8.0)
plt.rcParams['font.size'] = 16

### prepapre the Train/Val/Test split 
We are going to classify images from CIFAR10, let's first download the dataset

In [ ]:
tsfms = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda z: z.flatten())]) 
train_ds = torchvision.datasets.SVHN(root="/data/", split='train', transform=tsfms, download=True)
test_ds = torchvision.datasets.SVHN(root="/data/", split='test', transform=tsfms, download=True)

n_classes = 10
n_features = len(train_ds[0][0])

In [ ]:
f, axarr = plt.subplots(1,2, squeeze=False)
axarr[0,0].imshow(test_ds[0][0].reshape(3,32,32).permute(1,2,0))
axarr[0,1].imshow(train_ds[0][0].reshape(3,32,32).permute(1,2,0))

### We create a small validation set from the training set

In [ ]:
splitted_datasets = torch.utils.data.random_split(train_ds, [len(train_ds)-5000, 5000])
actual_train_subds = splitted_datasets[0]
valid_subds = splitted_datasets[1]

#### The data loader class

The class `torch.utils.data.DataLoader` represents a Python iterable over a dataset with support for automatic batching, multi-process data loading and many more features. The data loader communicates with the dataset using the function `__getitem__`, and stacks its outputs as tensors over the first dimension to form a batch.
In contrast to the dataset class, we usually don't have to define our own data loader class, but can just create an object of it with the dataset as input. Additionally, we can configure our data loader with the following input arguments (only a selection, see full list [here](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)):

* `batch_size`: Number of samples to stack per batch
* `shuffle`: If True, the data is returned in a random order. This is important during training for introducing stochasticity. 
* `num_workers`: Number of subprocesses to use for data loading. The default, 0, means that the data will be loaded in the main process which can slow down training for datasets where loading a data point takes a considerable amount of time (e.g. large images). More workers are recommended for those, but can cause issues on Windows computers. For tiny datasets as ours, 0 workers are usually faster.
* `pin_memory`: If True, the data loader will copy Tensors into CUDA pinned memory before returning them. This can save some time for large data points on GPUs. Usually a good practice to use for a training set, but not necessarily for validation and test to save memory on the GPU.
* `drop_last`: If True, the last batch is dropped in case it is smaller than the specified batch size. This occurs when the dataset size is not a multiple of the batch size. Only potentially helpful during training to keep a consistent batch size.

Let's create a simple data loader below:

In [ ]:
batch_size = 256

train_dl = torch.utils.data.DataLoader(actual_train_subds, batch_size=batch_size, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_subds, batch_size=batch_size)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=batch_size)

In [ ]:
# next(iter(...)) catches the first batch of the data loader
# If shuffle is True, this will return a different batch every time we run this cell
# For iterating over the whole dataset, we can simple use "for batch in data_loader: ..."
data_inputs, data_labels = next(iter(train_dl))

# The shape of the outputs are [batch_size, d_1,...,d_N] where d_1,...,d_N are the 
# dimensions of the data point returned from the dataset class
print("Data inputs", data_inputs.shape, "\n", data_inputs)
print("Data labels", data_labels.shape, "\n", data_labels)

### Create the network

We will then create our first Neural Network. One way to create Neural Networks in PyTorch is by subclassing `torch.nn.Module`. In this way, our model will inherit a lot of ready-to-use convinience functions (access to parameters for optimization, get/set parameters, ...). 

We only need to create the layers we will use in the `__init__` function and define the `forward` function that specifies how to apply them. 

Layers are in turn subclasses of `torch.nn.Module`. In our example, we will use only linear layers, i.e. Fully Connected (FC) layers. Our network will have at least two FC layers: `self.first`, mapping the flattened input image into the (first) hidden representation, and `self.last`, mapping the (last) hidden representation into the scores for the classes.

To play with varying depths and activation functions, we will have two additional parameters:


*   `n_additional_hidden_layers`, specifies how many hidden layers our network has, beside `self.first`
*   `use_relu`, if `False`, activations will be sigmoid functions, ReLUs otherwise

Note that to store a variable number of layers in our network, we do not use plain PyTorch lists, but `torch.nn.ModuleList`. This is important to make PyTorch aware of the layers in the list, e.g. to set/get their parameters when calling the methods of the base `Module` class.

In [ ]:
### Common template
class SimpleClassifier(nn.Module):

    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        return x

In [ ]:
class SimpleClassifier(torch.nn.Module):
  def __init__(self, n_features, hidden_width, n_classes, n_additional_hidden_layers=0):
    super(SimpleClassifier, self).__init__()
    self.first = torch.nn.Linear(n_features, hidden_width) 
    self.activation = torch.relu
    self.last = torch.nn.Linear(hidden_width, n_classes)

    self.additional_hidden_layers = torch.nn.ModuleList(
        [torch.nn.Linear(hidden_width, hidden_width) for i in range(n_additional_hidden_layers)])
  
      
  def forward(self, x):
    x = self.first(x)
    x = self.activation(x)
    for layer in self.additional_hidden_layers:
      x = layer(x)
      x = self.activation(x)
    x = self.last(x)
    return x

In [ ]:
hidden_width = 128

model = SimpleClassifier(n_features, hidden_width, n_classes, n_additional_hidden_layers=1)
for p in model.parameters():
  print(type(p), p.shape)

Each linear layer has a weight matrix of the shape `[output, input]`, and a bias of the shape `[output]`. The activation function does not have any parameters. Note that parameters are only registered for `nn.Module` objects that are direct object attributes, i.e. `self.a = ...`. 

`state_dict()` instead is an (Ordered) Dictionary, which associates each variable storing a layer in our classes with its parameters. It is therefore useful to obtain a snapshot of the parameters of our model that can later be restored by calling `load_state_dict()`.

In [ ]:
type(model.state_dict())

In [ ]:
model.state_dict().keys()

In [ ]:
model.state_dict()["first.bias"]

In [ ]:
# Push model to device. Has to be only done once
model.to(device)

### Optimization

After defining the model and the dataset, it is time to prepare the optimization of the model. During training, we will perform the following steps:

1. Get a batch from the data loader
2. Obtain the predictions from the model for the batch
3. Calculate the loss based on the difference between predictions and labels
4. Backpropagation: calculate the gradients for every parameter with respect to the loss
5. Update the parameters of the model in the direction of the gradients


#### Stochastic Gradient Descent

For updating the parameters, PyTorch provides the package `torch.optim` that has most popular optimizers implemented. Will for now use `torch.optim.Adam`.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

The optimizer provides two useful functions: `optimizer.step()`, and `optimizer.zero_grad()`. The step function updates the parameters based on the gradients as explained above. The function `optimizer.zero_grad()` sets the gradients of all parameters to zero. While this function seems less relevant at first, it is a crucial pre-step before performing backpropagation. If we call the `backward` function on the loss while the parameter gradients are non-zero from the previous batch, the new gradients would actually be added to the previous ones instead of overwriting them. This is done because a parameter might occur multiple times in a computation graph, and we need to sum the gradients in this case instead of replacing them. Hence, remember to call `optimizer.zero_grad()` before calculating the gradients of a batch.

In [ ]:
def ncorrect(scores, y):
  y_hat = torch.argmax(scores, 1)
  return (y_hat==y).sum()

def accuracy(scores, y):
  correct = ncorrect(scores, y)
  return correct.true_divide(y.shape[0])
  
def train_loop(model, train_dl, epochs, opt, valid_dl=None, verbose=False):
  best_valid_acc = 0
  best_params = []
  best_epoch = -1

  for e in tqdm(range(epochs)):
    model.train()
    #train
    train_loss = 0
    train_samples = 0
    train_acc = 0
    for train_data in train_dl:
      input = train_data[0].to(device)
      labels = train_data[1].to(device)
      scores = model(input)
      loss = F.cross_entropy(scores, labels)
      train_loss += loss.item() * input.shape[0]
      train_samples += input.shape[0]
      train_acc += ncorrect(scores, labels).item()
      loss.backward()

      opt.step()
      opt.zero_grad()

    train_acc /= train_samples
    train_loss /= train_samples
    
    # validation
    model.eval()
    with torch.no_grad():
      valid_loss = 0
      valid_samples = 0
      valid_acc = 0
      if valid_dl is not None:
        for valid_data in valid_dl:
          input = valid_data[0].to(device)
          labels = valid_data[1].to(device)
          valid_scores = model(input)
          valid_loss += F.cross_entropy(valid_scores, labels).item() * input.shape[0]
          valid_samples += input.shape[0]
          valid_acc += ncorrect(valid_scores, labels).item()
        valid_acc /= valid_samples
        valid_loss /= valid_samples
      
      if valid_dl is None or valid_acc > best_valid_acc:
        best_valid_acc = valid_acc if valid_dl is not None else 0
        best_params = model.state_dict()
        torch.save(best_params, "best_model.pth")
        best_epoch = e

    if verbose and e % 5 == 0:
      print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if valid_dl is None else f" - valid loss {valid_loss:.3f} - valid acc {valid_acc:.3f}"))
  
  if verbose and valid_dl is not None:
    print(f"Best epoch {best_epoch}, best acc {best_valid_acc}")

  return best_valid_acc, best_params, best_epoch

### Train the network

In [ ]:
epochs = 20

start = timer()
best_valid_acc, best_params, best_epoch = train_loop(model, train_dl, epochs, optimizer, valid_dl=valid_dl, verbose=True)
end = timer()
print(f"Elapsed time (s): {end-start}")

### Load the model 

In [ ]:
# Load state dict from the disk (make sure it is the same name as above)
state_dict = torch.load("best_model.pth")

# Create a new model and load the state
new_model = SimpleClassifier(n_features, hidden_width, n_classes, n_additional_hidden_layers=1)
new_model.load_state_dict(state_dict)

# Now it's your turn!

## Exercise 1: Improve the Accuracy score

Try to achieve a better accuracy on the validation set by modifying the previus model and manually tuning the hyper parameters such as the number of epochs, the learning rate, and the number of hidden layers. 


Suggestion: you can add [Batch Normalization](https://arxiv.org/abs/1502.03167). You can find the Normalization layers already implemented in [PyTorch](https://pytorch.org/docs/stable/nn.html#id1).

use '''nn.Sequential()''' to stack togheter a block with the following 
structure:

nn.Linear -> BatchNorm -> Activation

Use the nn.Sequential() pytorch package to stack the previus three components into a single layer.

**You should easily get 80% on the validation set**


In [ ]:
def get_linear_bn_relu(
    name: str,
    features_in: int,
    features_out: int,
    add_batch_norm: bool,
) -> nn.Sequential:
    """Gets a sequential container with a sandwich of Linear, Relu and batch norm.

    Args:
        name: the name prefix to append to each layer in the container.
        features_in: the number of the input channels.
        features_out: the number of the output channels.
        add_batch_norm: if True, add a batch normalization layer after the Linear layer.

    Returns: the created sequential.
    """
    container = nn.Sequential()
    ############ Write your code here ############

    #############################################


    return container

In [ ]:
class SimpleClassifierBN(nn.Module):
  def __init__(self, n_features, hidden_width, n_classes, n_additional_hidden_layers=0):
    super(SimpleClassifierBN, self).__init__()
    self.first = torch.nn.Linear(n_features, hidden_width)
    self.bn = nn.BatchNorm1d(hidden_width)
    self.activation = torch.relu
    self.last = torch.nn.Linear(hidden_width, n_classes)

    self.additional_hidden_layers = torch.nn.ModuleList(
        [get_linear_bn_relu(name=f"h_layer_{i}", features_in=hidden_width, features_out=hidden_width, add_batch_norm=True)
          for i in range(n_additional_hidden_layers)])
  
  def forward(self, x):
    x = self.first(x)
    x = self.bn(x)
    x = self.activation(x)
    for layer in self.additional_hidden_layers:
      x = layer(x)
      x = self.activation(x)
    x = self.last(x)
    return x

In [ ]:
new_model = SimpleClassifierBN(n_features, hidden_width, n_classes, n_additional_hidden_layers=3)
new_model.to(device)
optimizer = torch.optim.Adam(new_model.parameters(), lr=0.01)
epochs = 25

start = timer()
best_valid_acc, best_params, best_epoch = train_loop(new_model, train_dl, epochs, optimizer, valid_dl=valid_dl, verbose=True)
end = timer()
print(f"Elapsed time (s): {end-start}")

In [ ]:
new_model.load_state_dict(torch.load('best_model.pth'))

## Exercise 2: 
Compute the confusion matrix for the test set. Then, given the confusion matrix, compute the accuracy score.

In [ ]:
num_classes = 10
new_model.eval()
confusion_matrix = torch.zeros(num_classes, num_classes)
############ Write your code here ############
with torch.no_grad():
    for i, data in enumerate(test_dl):
        input, gt = data
        input = input.to(device)
        gt = gt.to(device)

test_accuracy = torch.diagonal(confusion_matrix).sum()/confusion_matrix.sum()
#############################################
print(test_accuracy)


In [ ]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

confusion_matrix /= confusion_matrix.sum(1)
confusion_matrix *= 100
df_cm = pd.DataFrame(confusion_matrix)
plt.figure(figsize = (10,7))
sn.heatmap(df_cm, annot=True)

## Exercise 3:
Rewrite the forward model to return the embedding rather than the classification scores.
**Note that you don't have to retrain the model again!**

Then, show the three nearest neighbors in the train set for element with index 0 in the test set.

In [ ]:
class SimpleClassifierBN_2(nn.Module):
  def __init__(self, n_features, hidden_width, n_classes, n_additional_hidden_layers=0):
    super(SimpleClassifierBN_2, self).__init__()
    ############ Write your code here ############

  #############################################

  def forward(self, x):
    ############ Write your code here ############

    #############################################

In [ ]:
# Load state dict from the disk (make sure it is the same name as above)
state_dict = torch.load("best_model.pth")

# Create a new model and load the state
new_model = SimpleClassifierBN_2(n_features, hidden_width, n_classes, n_additional_hidden_layers=3)
new_model.load_state_dict(state_dict)
new_model.to(device)
new_model.eval()

In [ ]:
def get_embeddings(model, dl):
  model.eval()
  embeddings = []
  labels = []
  
############ Write your code here ############
  with torch.no_grad():
      for data in dl:
          input, gt = data
          input = input.to(device)
          gt = gt.to(device)

          # calculate the embeddings by running images through the network and store them
          
#############################################
  return embeddings, labels

train_dl = torch.utils.data.DataLoader(actual_train_subds, batch_size=batch_size, shuffle=False)
embeddings_train, labels_train = get_embeddings(new_model, train_dl)
print(embeddings_train.shape, labels_train.shape)
embeddings_test, labels_test = get_embeddings(new_model, test_dl)
print(embeddings_test.shape, labels_test.shape)

In [ ]:
############ Write your code here ############

elem_idx = 0

#############################################
